<a href="https://colab.research.google.com/github/ilonajulczuk/podcast_takeways/blob/main/drive_transcriber.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q --upgrade torch torchvision torchaudio
!pip install -q transformers
!pip install -q ipython-autotime

In [2]:
%load_ext autotime

time: 296 µs (started: 2024-10-14 08:16:22 +00:00)


In [3]:
import torch
from transformers import pipeline

time: 23.7 s (started: 2024-10-14 08:16:22 +00:00)


In [4]:
pipe = pipeline("automatic-speech-recognition",
                "openai/whisper-large-v2",
                torch_dtype=torch.float16,
                device="cuda:0")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.17G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/4.29k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

time: 1min 15s (started: 2024-10-14 08:16:46 +00:00)


In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive
time: 1min 10s (started: 2024-10-14 08:18:01 +00:00)


In [6]:
%cd /content/drive/MyDrive/audiotranscriptions

/content/drive/MyDrive/audiotranscriptions
time: 7.07 ms (started: 2024-10-14 08:19:12 +00:00)


In [7]:
from pathlib import Path, PosixPath


def get_file_prefix(filename: PosixPath):
  return filename.name.split(".")[0]

def needs_transcription(audio: PosixPath, transcriptions: set[str]):
  return get_file_prefix(audio) not in transcriptions


files = Path.cwd().iterdir()
audios = [file for file in files if file.suffix in [".mp3", ".wav"]]
transcriptions = {get_file_prefix(file) for file in Path.cwd().iterdir() if file.suffix == ".txt"}

print("audios:", audios)
print("transcriptions:", transcriptions)

audios: [PosixPath('/content/drive/MyDrive/audiotranscriptions/harris_1.mp3'), PosixPath('/content/drive/MyDrive/audiotranscriptions/gottmans.mp3'), PosixPath('/content/drive/MyDrive/audiotranscriptions/crying.mp3'), PosixPath('/content/drive/MyDrive/audiotranscriptions/immature_people.mp3')]
transcriptions: {'immature_people', 'gottmans', 'harris_1', 'crying'}
time: 201 ms (started: 2024-10-14 08:19:12 +00:00)


In [8]:
for audio in audios:
  print(f"Audio: {audio.name} needs transcription: {needs_transcription(audio, transcriptions)}")

Audio: harris_1.mp3 needs transcription: False
Audio: gottmans.mp3 needs transcription: False
Audio: crying.mp3 needs transcription: False
Audio: immature_people.mp3 needs transcription: False
time: 704 µs (started: 2024-10-14 08:19:12 +00:00)


In [9]:
def transcribe_file(pipe, filename, transcription_filename):
  print("transcribing", filename)
  outputs = pipe(filename,
                chunk_length_s=30,
                batch_size=16,
                return_timestamps=True)
  text = outputs["text"]

  with open(transcription_filename, "w") as f:
    f.write(text)

time: 437 µs (started: 2024-10-14 08:19:12 +00:00)


In [10]:
def transcribe_all(pipe, audios, transcriptions):
  for audio in audios:
    if needs_transcription(audio, transcriptions):
      transcription_filename = audio.with_suffix(".txt")
      transcribe_file(pipe, audio.name, transcription_filename)
      transcriptions.add(get_file_prefix(audio))

time: 387 µs (started: 2024-10-14 08:19:12 +00:00)


In [11]:
transcribe_all(pipe, audios, transcriptions)

time: 341 µs (started: 2024-10-14 08:19:12 +00:00)


time: 9.18 ms (started: 2024-10-14 08:19:12 +00:00)
